In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
class MulLayer:
    def __init__(self):
        self.x=None
        self.y=None

    def forward(self, x, y):
        self.x=x
        self.y=y
        out=x*y

        return out

    def backward(self, dout): #dout는 미분
        dx=dout*self.y #x와 y를 바꾼다.
        dy=dout*self.x

        return dx, dy

In [3]:
apple=100
apple_num=2
tax=1.1

mul_apple_layer=MulLayer()
mul_tax_layer=MulLayer()

apple_price=mul_apple_layer.forward(apple, apple_num)
price=mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [4]:
dprice=1
dapple_price, dtax=mul_tax_layer.backward(dprice)
dapple, dapple_num=mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


In [5]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x,y):
        out=x+y

        return out

    def backward(self, dout):
        dx=dout*1
        dy=dout*1

        return dx, dy

In [6]:
apple=100
apple_num=2
orange=150
orange_num=3
tax=1.1

mul_apple_layer=MulLayer()
mul_orange_layer=MulLayer()
add_apple_orange_layer=AddLayer()
mul_tax_layer=MulLayer()

#순전파
apple_price=mul_apple_layer.forward(apple, apple_num)
orange_price=mul_orange_layer.forward(orange, orange_num)
all_price=add_apple_orange_layer.forward(apple_price, orange_price)
price=mul_tax_layer.forward(all_price, tax)

#역전파
dprice=1
dall_price, dtax=mul_tax_layer.backward(dprice)
dapple_price, dorange_price=add_apple_orange_layer.backward(dall_price)
dorange, dorange_num=mul_orange_layer.backward(dorange_price)
dapple, dapple_num=mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


In [7]:
class Relu:
    def __init__(self):
        self.mask=None

    def forward(self, x):
        self.mask=(x<=0)
        out=x.copy()
        out[self.mask]=0

        return out

    def backward(self, dout):
        dout[self.mask]=0
        dx=dout

        return dx

In [8]:
x=np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [9]:
mask=(x<=0)
print(mask)

[[False  True]
 [ True False]]


In [10]:
class Sigmoid:
    def __init__(self):
        self.out=None

    def forward(self, x):
        out=1/(1+np.exp(-x))
        self.out=out

        return out

    def backward(self, dout):
        dx=dout*(1.0-self.out)*self.out

        return dx

In [11]:
X_dot_W=np.array([[0,0,0],[10,10,10]])
B=np.array([1,2,3])
print(X_dot_W+B)

[[ 1  2  3]
 [11 12 13]]


In [12]:
dY=np.array([[1,2,3],[4,5,6]])
print(dY)

dB=np.sum(dY,axis=0)
print(dB)

[[1 2 3]
 [4 5 6]]
[5 7 9]


In [13]:
class Affine:
    def __init__(self, W, b):
        self.W=W
        self.b=b
        self.x=None
        self.dW=None
        self.db=None

    def forward(self, x):
        self.x=x
        out=np.dot(x,self.W)+self.b

        return out

    def backward(self, dout):
        dx=np.dot(dout, self.W.T)
        self.dW=np.dot(self.x.T, dout)
        self.db=np.sum(dout, axis=0)

        return dx

In [14]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss=None #손실
        self.y=None #softmax의 출력
        self.t=None #정답 레이블(원-핫 코딩)

    def forward(self, x, t):
        self.t=t
        self.y=softmax(x)
        self.loss=cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size=self.t.shape[0]
        dx=(self.y-self.t)/batch_size

        return dx

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
import sys,os
sys.path.append('/content/drive/MyDrive/연구실/밑바닥부터 시작하는 딥러닝/common')

In [17]:
import numpy as np
from layers import *
from gradient import numerical_gradient
from collections import OrderedDict

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params={}
        self.params['W1']=weight_init_std*np.random.randn(input_size, hidden_size)
        self.params['b1']=np.zeros(hidden_size)
        self.params['W2']=weight_init_std*np.random.rand(hidden_size, output_size)
        self.params['b2']=np.zeros(output_size)

        self.layers=OrderedDict()
        self.layers['Affine1']=Affine(self.params['W1'], self.parmas['b1'])
        self.layers['Relu1']=Relu()
        self.layers['Affine2']=Affine(self.params['W2'], self.params['b2'])

        self.lastLayer=SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x=layer.forward()

        return x

        self.layers['Affine1']=Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1']=Relu()
        self.layers['Affine2']=Affine(self.params['W2'], self.params['b2'])

        self.lastLayer=SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x=layer.forward(x)

        return x

    def loss(self, x, t):
        y=self.predict(x)
        return self.lastLayer.forward(y,t)

    def accuracy(self, x, t):
        y=self.predict(x)
        y=np.argmax(y, axis=1)
        if t.ndim!=1:
            t=np.argmax(t, axis=1)

        accuracy=np.sum(y==t) / float(x.shape[0])
        return accuracy

    def numerical_gradient(self, x, t):
        loss_W=lambda W:self.loss(x,t)

        grads={}
        grads['W1']=numerical_gradient(loss_W, self.params['W1'])
        grads['b1']=numerical_gradient(loss_W, self.params['b1'])
        grads['W2']=numerical_gradient(loss_W, self.params['W2'])
        grads['b2']=numerical_gradient(loss_W, self.params['b2'])

    def gradient(self, x, t):
        self.loss(x,t)

        dout=1
        dout=self.lastLayer.backward(dout)

        layers=list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout=layer.backward(dout)

        grads={}
        grads['W1']=self.layers['Affine1'].dW
        grads['b1']=self.layers['Affine1'].db
        grads['W2']=self.layers['Affine2'].dW
        grads['b2']=self.layers['Affine2'].db

        return grads

In [19]:
sys.path.append('/content/drive/MyDrive/연구실/밑바닥부터 시작하는 딥러닝/dataset')

In [20]:
from two_layer_net import TwoLayerNet
from mnist import load_mnist

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
(x_train, t_train),(x_test, t_test)=load_mnist(normalize=True, flatten=True, one_hot_label=True)

network=TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch=x_train[:3]
t_batch=t_train[:3]

grad_numerical=network.numerical_gradient(x_batch, t_batch)
grad_backprop=network.gradient(x_batch, t_batch)

for key in grad_numerical.keys():
    diff=np.average(np.abs(grad_backprop[key]-grad_numerical[key]))
    print(key+":"+str(diff))

W1:1.9365696908222647e-10
b1:9.161735802166093e-10
W2:7.083748064183603e-08
b2:1.400502470522147e-07


In [23]:
print(x_train.shape)
print(t_train.shape)

(60000, 784)
(60000,)


In [22]:
print(x_batch.shape)
print(t_batch.shape)

(3, 784)
(3,)


In [27]:
iters_num=10000
train_size=x_train.shape[0]
batch_size=100
learning_rate=0.1

train_loss_list=[]
train_acc_list=[]
test_acc_list=[]

iter_per_epoch=max(train_size/batch_size, 1)

for i in range(iters_num):
    batch_mask=np.random.choice(train_size, batch_size)
    x_batch=x_train[batch_mask]
    t_batch=t_train[batch_mask]

    grad=network.gradient(x_batch, t_batch)

    for key in ('W1','b1','W2','b2'):
        network.params[key]-=learning_rate*grad[key]

    loss=network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i%iter_per_epoch==0:
        train_acc=network.accuracy(x_train, t_train)
        test_acc=network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.10218333333333333 0.101
0.7769 0.784
0.8766666666666667 0.8808
0.8993 0.9029
0.9075 0.9117
0.91515 0.917
0.9205166666666666 0.9219
0.9238666666666666 0.9278
0.9277 0.9304
0.9314333333333333 0.9331
0.9339 0.9345
0.9377833333333333 0.9376
0.9396166666666667 0.9395
0.9406666666666667 0.9413
0.9430333333333333 0.943
0.945 0.9443
0.9468833333333333 0.9463
